# Introduction

QA plots for the INTT detector

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for refer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_new/new.4`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In file included from input_line_59:4:
/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_new/new.4/rootmacros/sPhenixStyle.C:15:6: error: redefinition of 'SetsPhenixStyle'
void SetsPhenixStyle()
     ^
/sphenix/user/jdosbo/sPhenixStyle.C:55:6: note: previous definition is here
void SetsPhenixStyle ()
     ^
In file included from input_line_59:4:
/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_new/new.4/rootmacros/sPhenixStyle.C:24:9: error: redefinition of 'sPhenixStyle'
TStyle* sPhenixStyle()
        ^
/sphenix/user/jdosbo/sPhenixStyle.C:64:9: note: previous definition is here
TStyle* sPhenixStyle() 
        ^


In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.


In [9]:
%jsroot on

## Inputs and file checks

In [10]:
qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))

qa_file_name_ref = os.getenv("qa_file_name_ref")
if qa_file_name_ref is None:
    qa_file_name_ref = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_ref` env not set. use the default `qa_file_name_ref={qa_file_name_ref}`"))
elif qa_file_name_ref == 'None':
    qa_file_name_ref = None
    display(Markdown(f"`qa_file_name_ref` = None and we are set to not to use the reference histograms"))
    

display(Markdown(f"Opening QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

if qa_file_name_ref is not None:
#     qa_file_ref = ROOT.TFile.Open(qa_file_name_ref);

#     assert qa_file_ref.IsOpen()
    display(Markdown(f"Opening QA reference file at `{qa_file_name_ref}`"))
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = new TFile(\"{qa_file_name_ref}\");")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = \"{qa_file_name_ref}\";")
else:
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = nullptr;")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = nullptr;")

`qa_file_name_new` env not set. use the default `qa_file_name_new=prdf_reconstruction53877_qa.root`

`qa_file_name_ref` env not set. use the default `qa_file_name_ref=prdf_reconstruction53877_qa.root`

Opening QA file at `prdf_reconstruction53877_qa.root`

Opening QA reference file at `prdf_reconstruction53877_qa.root`

## Raw Hit quantities

In [11]:
%%cpp

//TFile *qa_file_new = TFile::Open("prdf_reconstruction53877_qa.root");
if(qa_file_new == nullptr)
{
    std::cout << "Failed to open file " << qa_file_name_new << std::endl;
    exit(1);
}


In [12]:
%%cpp
    TCanvas* m_canvas = new TCanvas("m_canvas", "INTT Hits Summary", 800, 600);
    m_canvas->Divide(2, 3);

    // Get histograms
    TH1* new_nhit = dynamic_cast<TH1*>(qa_file_new->Get("h_InttRawHitQA_nhit"));
    TH1* new_nhitnorth = dynamic_cast<TH1*>(qa_file_new->Get("h_InttRawHitQA_nhit_north"));
    TH1* new_nhitsouth = dynamic_cast<TH1*>(qa_file_new->Get("h_InttRawHitQA_nhit_south"));
    TH1* new_pid = dynamic_cast<TH1*>(qa_file_new->Get("h_InttRawHitQA_pid"));
    TH1* new_adc = dynamic_cast<TH1*>(qa_file_new->Get("h_InttRawHitQA_adc"));
    TH1* new_bco = dynamic_cast<TH1*>(qa_file_new->Get("h_InttRawHitQA_bco"));
    TH1* new_bcofull = dynamic_cast<TH1*>(qa_file_new->Get("h_InttRawHitQA_bco_full"));
    
    TH1* ref_nhit = dynamic_cast<TH1*>(qa_file_ref->Get("h_InttRawHitQA_nhit"));
    TH1* ref_nhitnorth = dynamic_cast<TH1*>(qa_file_ref->Get("h_InttRawHitQA_nhit_north"));
    TH1* ref_nhitsouth = dynamic_cast<TH1*>(qa_file_ref->Get("h_InttRawHitQA_nhit_south"));
    TH1* ref_pid = dynamic_cast<TH1*>(qa_file_ref->Get("h_InttRawHitQA_pid"));
    TH1* ref_adc = dynamic_cast<TH1*>(qa_file_ref->Get("h_InttRawHitQA_adc"));
    TH1* ref_bco = dynamic_cast<TH1*>(qa_file_ref->Get("h_InttRawHitQA_bco"));
    TH1* ref_bcofull = dynamic_cast<TH1*>(qa_file_ref->Get("h_InttRawHitQA_bco_full"));
    
    
    // Draw histograms to canvas
    if (new_nhit && ref_nhit) {
        m_canvas->cd(1);
        new_nhit->GetXaxis()->SetTitle("N_{hit}");
        
        DrawReference(new_nhit, ref_nhit);
    } else {
        std::cerr << "Histogram nhit not found" << std::endl;
    }

    if (new_nhitnorth && ref_nhitnorth) {
        m_canvas->cd(2);
        new_nhitnorth->GetYaxis()->SetTitle("N_{hit}");
        DrawReference(new_nhitnorth, ref_nhitnorth);
    } else {
        std::cerr << "Histogram nhitnorth not found" << std::endl;
    }

    if (new_nhitsouth && ref_nhitsouth) {
        m_canvas->cd(3);
        new_nhitsouth->GetYaxis()->SetTitle("N_{hit}");
        DrawReference(new_nhitsouth, ref_nhitsouth);
    } else {
        std::cerr << "Histogram nhitsouth not found" << std::endl;
    }

    if (new_pid && ref_pid) {
        m_canvas->cd(4);
        DrawReference(new_pid, ref_pid);
    } else {
        std::cerr << "Histogram pid not found" << std::endl;
    }

    if (new_adc && ref_adc) {
        m_canvas->cd(5);
        DrawReference(new_adc, ref_adc);
    } else {
        std::cerr << "Histogram adc not found" << std::endl;
    }

    if (new_bco && ref_bco) {
        m_canvas->cd(6);
        new_bco->GetXaxis()->SetTitle("Crossing");
        DrawReference(new_bco, ref_bco);
    } else {
        std::cerr << "Histogram bco not found" << std::endl;
    }

  



    // Update and save the canvas
    m_canvas->Update();





input_line_73:2:3: error: use of undeclared identifier 'DrawReference'
 (DrawReference(((*(TH1 **)0x7f7bd966f010)), ((*(TH1 **)0x7f7bd966f048))))
  ^
Error in <HandleInterpreterException>: Error evaluating expression (DrawReference(((*(TH1 **)0x7f7bd966f010)), ((*(TH1 **)0x7f7bd966f048))))
Execution of your code was aborted.


In [13]:
%%cpp




    TCanvas* m_canvas_chip = new TCanvas("m_canvas_chip", "INTT Plots 0", 1000, 800);
    m_canvas_chip->Divide(2, 2);
  
    // this one is used to plot the run number on the canvas
    TPad* transparent = new TPad("transparent_0", "this does not show", 0, 0, 1, 1);
    transparent->SetFillStyle(4000);
    transparent->Draw();
    
    // Get histograms
    TH1F* new_occupancy = dynamic_cast<TH1F*>(qa_file_new->Get("h_InttClusterQA_sensorOccupancy"));
    TH1F* new_clusSize = dynamic_cast<TH1F*>(qa_file_new->Get("h_InttClusterQA_clusterSize"));
    TH1F *new_clusterphi = dynamic_cast<TH1F*>(qa_file_new->Get("h_InttClusterQA_clusterPhi_incl"));
     TH1F* ref_occupancy = dynamic_cast<TH1F*>(qa_file_ref->Get("h_InttClusterQA_sensorOccupancy"));
    TH1F* ref_clusSize = dynamic_cast<TH1F*>(qa_file_ref->Get("h_InttClusterQA_clusterSize"));
    TH1F *ref_clusterphi = dynamic_cast<TH1F*>(qa_file_ref->Get("h_InttClusterQA_clusterPhi_incl"));
    // Draw histograms to canvas
    m_canvas_chip->cd(1);
    if (new_occupancy && ref_occupancy) {
        DrawReference(new_occupancy, ref_occupancy);
    } else {
        std::cerr << "Histogram h_occupancy not found" << std::endl;
        return;
    }

    m_canvas_chip->cd(2);
    if (new_clusSize && ref_clusSize) {
        DrawReference(new_clusSize, ref_clusSize);
    } else {
        std::cerr << "Histogram h_clusSize not found" << std::endl;
        return;
    }
    
    m_canvas_chip->cd(3);
    if(new_clusterphi && ref_clusterphi)
    {
        DrawReference(new_clusterphi, ref_clusterphi);
    }
    else
    {
        std::cerr << "Histogram h_clusterphi not found" << std::endl;
    }


    m_canvas_chip->Update();
 
    std::cout << "DrawChipInfo completed" << std::endl;


input_line_77:2:3: error: use of undeclared identifier 'DrawReference'
 (DrawReference(((*(TH1F **)0x7f7bd8002020)), ((*(TH1F **)0x7f7bd8002038))))
  ^
Error in <HandleInterpreterException>: Error evaluating expression (DrawReference(((*(TH1F **)0x7f7bd8002020)), ((*(TH1F **)0x7f7bd8002038))))
Execution of your code was aborted.
